In [2]:
from odyssey.core.bigquery.BigQueryGithubEntry import *
from odyssey.core.bigquery.GithubPython import GithubPython
from odyssey.core.analyzer import InstantiationAnalyzer
from odyssey.core.analyzer import ImportAnalyzer
import matplotlib.pyplot as plt
import numpy as np
from nbconvert import PythonExporter
import pandas as pd
import pickle
import ast
import jedi

import csv
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

from odyssey.core.bigquery.GithubPython import GithubPython as ghp
%matplotlib notebook 

In [103]:
class FunctionCallVisitor(ast.NodeVisitor):
    def visit_Call(self, node):
        print (ast.dump(node))

In [104]:
functionsList=['RF', 'GridSearchCV','make_pipeline','AdaBoostRegressor']

In [ ]:
query = """\
SELECT
    DISTINCT(repo_name)
FROM
    `Odyssey_github_sklearn.content_py_full` 
WHERE
    (REGEXP_CONTAINS(path,"sklearn")) OR
    (REGEXP_CONTAINS(repo_name,"scikit-learn")) OR
    (REGEXP_CONTAINS(repo_name,"sklearn")) OR
    (REGEXP_CONTAINS(path,"scikit-learn"))
"""
gp = GithubPython()



excludedRepos = [repo_name[0] for repo_name in gp.run(query, project="odyssey-193217")]
print(excludedRepos)

In [ ]:
def excludeByRepoFull(excludeByRepos):
    if not excludedRepos:
        return ""
    s = '(NOT REGEXP_CONTAINS(repo_name,"%s"))' % excludedRepos[0]
    for repo in excludedRepos[1:]:
        s += ' AND '
        s += '(NOT REGEXP_CONTAINS(repo_name,"%s"))' % repo
    return s

In [3]:
functionListFromFile= pd.read_csv("classNames.csv")
functionListFromFile.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
functionListFromFile = np.array(functionListFromFile)
for file in functionListFromFile:
    print(file[0])

ARDRegression
AdaBoostClassifier
AdaBoostRegressor
AdditiveChi2Sampler
AffinityPropagation
AgglomerativeClustering
BaggingClassifier
BaggingRegressor
BayesianGaussianMixture
BayesianRidge
BernoulliNB
BernoulliRBM
Binarizer
Birch
CCA
CalibratedClassifierCV
CheckingClassifier
ClassifierChain
CountVectorizer
DBSCAN
DPGMM
DecisionTreeClassifier
DecisionTreeRegressor
DictVectorizer
DictionaryLearning
DummyClassifier
DummyRegressor
ElasticNet
ElasticNetCV
EllipticEnvelope
EmpiricalCovariance
ExtraTreeClassifier
ExtraTreeRegressor
ExtraTreesClassifier
ExtraTreesRegressor
FactorAnalysis
FastICA
FeatureAgglomeration
FeatureHasher
FeatureUnion
FunctionTransformer
GMM
GaussianMixture
GaussianNB
GaussianProcess
GaussianProcessClassifier
GaussianProcessRegressor
GaussianRandomProjection
GaussianRandomProjectionHash
GenericUnivariateSelect
GradientBoostingClassifier
GradientBoostingRegressor
GraphLasso
GraphLassoCV
GridSearchCV
GridSearchCV
HashingVectorizer
HuberRegressor
Imputer
IncrementalPCA
Iso

## All data from py_full

In [ ]:
AllContentQuery=  """\
    SELECT repo_name, content
    FROM  `Odyssey_github_sklearn.content_py_full`
    WHERE (REGEXP_CONTAINS(content, "from sklearn+ ") OR REGEXP_CONTAINS(content, "import sklearn+ ")) AND %s 
    GROUP BY 1,2
    """ %excludeByRepoFull(excludedRepos)

AllContent = gp.run(AllContentQuery)

AllContent_df = []

for i in range(len(AllContent)):
    AllContent_df.append((AllContent[i][0],AllContent[i][1]))
    
AllContent_df = pd.DataFrame(AllContent_df, columns= ['repo','content'])
AllContent_df.to_csv('AllContent_df.csv', encoding='utf-8', index=False)

In [5]:
AllContent_df=pd.read_csv('AllContent_df.csv', encoding ='utf-8')
AllContent_py_unique_df = pd.read_csv('AllContent_unique.csv', encoding='utf-8')
AllContent_ipynb_df =pd.read_csv('AllContent_ipynb_df.csv', encoding='utf-8')

## All data from py_unique

In [ ]:
AllContentQuery_unique=  """\
    SELECT repo_name, content
    FROM  `Odyssey_github_sklearn.content_py_unique`
    WHERE (REGEXP_CONTAINS(content, "from sklearn+ ") OR REGEXP_CONTAINS(content, "import sklearn+ ")) AND %s 
    GROUP BY 1,2
    """ %excludeByRepoFull(excludedRepos)

AllContent_unique = gp.run(AllContentQuery_unique)

AllContent_py_unique_df = []

for i in range(len(AllContent_unique)):
    AllContent_py_unique_df.append((AllContent_unique[i][0],AllContent_unique[i][1]))
    
AllContent_py_unique_df = pd.DataFrame(AllContent_py_unique_df, columns= ['repo','content'])
AllContent_py_unique_df.to_csv('AllContent_unique.csv', encoding='utf-8', index=False)

In [ ]:
AllContent_py_unique_df.head()

In [ ]:
query_ipynb = """\
SELECT
    DISTINCT(sample_repo_name)
FROM
    `Odyssey_github_sklearn.content_ipynb` 
WHERE
    (REGEXP_CONTAINS(sample_path,"sklearn")) OR
    (REGEXP_CONTAINS(sample_repo_name,"scikit-learn")) OR
    (REGEXP_CONTAINS(sample_repo_name,"sklearn")) OR
    (REGEXP_CONTAINS(sample_path,"scikit-learn"))
"""
gp = GithubPython()



excludedRepos_ipynb = [sample_repo_name[0] for sample_repo_name in gp.run(query_ipynb, project="odyssey-193217")]
print(excludedRepos_ipynb)

In [ ]:
def excludeByRepoFull_ipynb(excludeByRepos):
    if not excludedRepos:
        return ""
    s = '(NOT REGEXP_CONTAINS(sample_repo_name,"%s"))' % excludedRepos[0]
    for repo in excludedRepos[1:]:
        s += ' AND '
        s += '(NOT REGEXP_CONTAINS(sample_repo_name,"%s"))' % repo
    return s

In [ ]:
AllContentQuery_ipynb=  """\
    SELECT sample_repo_name, content
    FROM  `Odyssey_github_sklearn.content_ipynb`
    WHERE (REGEXP_CONTAINS(content, "from sklearn+ ") OR REGEXP_CONTAINS(content, "import sklearn+ ")) AND %s 
    GROUP BY 1,2
    """ %excludeByRepoFull_ipynb(excludedRepos_ipynb)

AllContent_ipynb = gp.run(AllContentQuery_ipynb)

AllContent_ipynb_df = []

for i in range(len(AllContent_ipynb)):
    AllContent_ipynb_df.append((AllContent_ipynb[i][0],AllContent_ipynb[i][1]))
    
AllContent_ipynb_df = pd.DataFrame(AllContent_ipynb_df, columns=['repo','content'])
AllContent_ipynb_df.to_csv('AllContent_ipynb_df.csv', encoding='utf-8', index=False)


In [ ]:
AllContent_ipynb_df.head()

In [ ]:
print(AllContent_ipynb_df.repo[1])
print("\n\n\n")
print(AllContent_ipynb_df.content[2])

In [ ]:
print(AllContent_py_unique_df.repo[989])
print("\n\n\n")
print(AllContent_py_unique_df.content[989])

## Import analyser

## Instantiation analyser for py_unique

In [139]:
hyperparameter_py_unique = dict()

for i in tqdm(range(len(AllContent_py_unique_df))):
    
    
            
    try:
        for file in functionListFromFile:
            
            if str(file[0]) not in hyperparameter_py_unique.keys():
                hyperparameter_py_unique[str(file[0])]={}
                    
                    
            analyzer= InstantiationAnalyzer(str(file[0]))
            analyzer.parse(AllContent_py_unique_df.content[i])

            if(len(analyzer.d.keys())!=0):
                
                for key in analyzer.d.keys():
                    
                    if key not in hyperparameter_py_unique[str(file[0])].keys():
                        hyperparameter_py_unique[str(file[0])][key]={}
                    
                    for subkey, subval in analyzer.d[key].items():
                        
                                             
                        if subkey in hyperparameter_py_unique[str(file[0])][key].keys():
                            hyperparameter_py_unique[str(file[0])][key][subkey] += subval
                        else:
                            hyperparameter_py_unique[str(file[0])][key][subkey] = subval
                        
                        
    except AttributeError:
        continue    
    

100%|██████████| 9246/9246 [28:25:53<00:00, 11.07s/it]   


In [140]:
## Saving as a pickle file

with open('hyperparameter_py_unique.pickle', 'wb') as handle:
    pickle.dump(hyperparameter_py_unique, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [7]:
## Reading the pickle file
with open('hyperparameter_py_unique.pickle', 'rb') as handle:
    hyperparameter_py_unique = pickle.load(handle)

## Instantiation analyser for ipynb files

### Dropping all json ipynb to py files

In [ ]:
for i in range(len(AllContent_ipynb_df)):
    
    file_name = "ipython_files/file"+str(i)+".ipynb "
    print(file_name)


In [4]:

excepted_files = []
for i in tqdm(range(len(AllContent_ipynb_df))):
    
    imp_file_name= "ipython_files/file"+str(i)+".ipynb"
    exp_file_name= "ipython_files/file"+str(i)+".py"
    
    python_exp = PythonExporter()
    
    try:
  
        data = python_exp.from_file(imp_file_name)

        with open(exp_file_name, 'w',encoding='utf-8') as file:
            file.write(str(data))
            
            
    except:
        excepted_files.append((imp_file_name+"\n"))
        continue
    
              
            

  3%|▎         | 129/4404 [00:37<20:30,  3.48it/s]--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\aishg\Anaconda3\lib\site-packages\nbformat\validator.py", line 258, in validate
    return validator.validate(nbjson)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\jsonschema\validators.py", line 130, in validate
    raise error
jsonschema.exceptions.ValidationError: {'cell_type': 'code', 'metadata': {'collapsed': False}, 'outputs': [{'ename': 'KeyboardInterrupt', 'evalue': '', 'output_type': 'error', 'traceback': ['\x1b---------------------------------------------------------------------------\x1b\n\x1bKeyboardInterrupt\x1b                         Traceback (most recent call last)', '\x1b<ipython-input-239-1ed456eed357>\x1b in \x1b<module>\x1b()\x1b\n\x1b----> 1\x1b \x1bsns\x1b\x1b.\x1b\x1bdistplot\x1b\x1b(\x1b\x1by0\x1b\x1b,\x1b \x1bkde\x1b\x1b=\x1b\x1bFalse\x1b\x1b,\x1b \x1brug\x1b\x1b=\x1b\x1bTrue\x1b\x1b)\x1b\x1b\x1b\x1b\n\x1b', '\x1b/usr/local/lib/pytho


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\aishg\Anaconda3\lib\site-packages\nbconvert\exporters\exporter.py", line 313, in _preprocess
    nbformat.validate(nbc, relax_add_props=True)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\nbformat\validator.py", line 260, in validate
    raise better_validation_error(e, version, version_minor)
nbformat.validator.NotebookValidationError: '*' is not of type 'integer', 'null'

Failed validating 'type' in code_cell['properties']['execution_count']:

On instance['cells'][77]['execution_count']:
'*'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\aishg\Anaconda3\lib\logging\__init__.py", line 992, in emit
    msg = self.format(record)
  File "C:\Users\aishg\Anaconda3\lib\logging\__init__.py", line 838, in format
    return fmt.format(record)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\tr

  File "C:\Users\aishg\Anaconda3\lib\site-packages\tornado\ioloop.py", line 888, in start
    handler_func(fd_obj, events)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\tornado\stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\zmq\eventloop\zmqstream.py", line 440, in _handle_events
    self._handle_recv()
  File "C:\Users\aishg\Anaconda3\lib\site-packages\zmq\eventloop\zmqstream.py", line 472, in _handle_recv
    self._run_callback(callback, msg)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\zmq\eventloop\zmqstream.py", line 414, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\tornado\stack_context.py", line 277, in null_wrapper
    return fn(*args, **kwargs)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "C:\Users\aishg\Anaconda3\lib\site-package

  File "C:\Users\aishg\Anaconda3\lib\site-packages\ipykernel\zmqshell.py", line 533, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2728, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2850, in run_ast_nodes
    if self.run_code(code, result):
  File "C:\Users\aishg\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-c7b788552763>", line 12, in <module>
    data = python_exp.from_file(imp_file_name)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\nbconvert\exporters\exporter.py", line 192, in from_file
    return self.from_notebook_node(nbformat.read(file_stream, as_version=4), resources=resources, **kw)
  File "C:\Users\aishg\

 12%|█▏        | 550/4404 [02:55<20:28,  3.14it/s]--- Logging error ---
Traceback (most recent call last):
  File "C:\Users\aishg\Anaconda3\lib\site-packages\nbformat\validator.py", line 258, in validate
    return validator.validate(nbjson)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\jsonschema\validators.py", line 130, in validate
    raise error
jsonschema.exceptions.ValidationError: {'cell_type': 'code', 'metadata': {'collapsed': False}, 'outputs': [], 'source': 'def insert_vectorized_line(corpus, labels, line, label, vectors_size, corpus_number_perdomain):\n    """\n    From a line: "word1 word2 word3 word4 word5 word6" and vectors_size = 3\n    return     : ["word1 word2 word3","word4 word5 word6"]\n    \n    The return DS is the format to vectorizer (fit_transform) list of words into vectors\n    """\n    # Get list of words from the line ["w1", "w2" .... ]\n    words =  re.findall(r\'\\b[a-z]+\\b\', line)\n    if len(words) > 0:\n        # Create vectors of length = vect

...

  File "C:\Users\aishg\Anaconda3\lib\site-packages\nbformat\validator.py", line 260, in validate
    raise better_validation_error(e, version, version_minor)
nbformat.validator.NotebookValidationError: '*' is not of type 'integer', 'null'

Failed validating 'type' in code_cell['properties']['execution_count']:

On instance['cells'][49]['execution_count']:
'*'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\aishg\Anaconda3\lib\logging\__init__.py", line 992, in emit
    msg = self.format(record)
  File "C:\Users\aishg\Anaconda3\lib\logging\__init__.py", line 838, in format
    return fmt.format(record)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\traitlets\config\application.py", line 117, in format
    return super(LevelFormatter, self).format(record)
  File "C:\Users\aishg\Anaconda3\lib\logging\__init__.py", line 575, in format
    record.message = record.getMessage()
  File "C:\Users\aishg\Anaconda3\lib\l


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\aishg\Anaconda3\lib\site-packages\nbconvert\exporters\exporter.py", line 313, in _preprocess
    nbformat.validate(nbc, relax_add_props=True)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\nbformat\validator.py", line 260, in validate
    raise better_validation_error(e, version, version_minor)
nbformat.validator.NotebookValidationError: 'execution_count' is a required property

Failed validating 'required' in execute_result:

On instance['cells'][21]['outputs'][0]:
{'data': {'text/html': '<div>\n'
                       '<table border="1" class="dataframe">\n'
                       '  <thead>\n'
                       '    <tr sty...'},
 'metadata': {},
 'output_type': 'execute_result'}

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\aishg\Anaconda3\lib\logging\__init__.py", line 992, in


During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\aishg\Anaconda3\lib\site-packages\nbconvert\exporters\exporter.py", line 313, in _preprocess
    nbformat.validate(nbc, relax_add_props=True)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\nbformat\validator.py", line 260, in validate
    raise better_validation_error(e, version, version_minor)
nbformat.validator.NotebookValidationError: 'execution_count' is a required property

Failed validating 'required' in execute_result:

On instance['cells'][7]['outputs'][0]:
{'data': {'text/html': '<div>\n'
                       '<table border="1" class="dataframe">\n'
                       '  <thead>\n'
                       '    <tr sty...'},
 'metadata': {},
 'output_type': 'execute_result'}

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\aishg\Anaconda3\lib\logging\__init__.py", line 992, in 

  File "C:\Users\aishg\Anaconda3\lib\logging\__init__.py", line 575, in format
    record.message = record.getMessage()
  File "C:\Users\aishg\Anaconda3\lib\logging\__init__.py", line 338, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Users\aishg\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\aishg\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\aishg\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\aishg\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\aishg\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    su

  File "C:\Users\aishg\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 235, in dispatch_shell
    handler(stream, idents, msg)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\ipykernel\kernelbase.py", line 399, in execute_request
    user_expressions, allow_stdin)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\ipykernel\ipkernel.py", line 196, in do_execute
    res = shell.run_cell(code, store_history=store_history, silent=silent)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\ipykernel\zmqshell.py", line 533, in run_cell
    return super(ZMQInteractiveShell, self).run_cell(*args, **kwargs)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2728, in run_cell
    interactivity=interactivity, compiler=compiler, result=result)
  File "C:\Users\aishg\Anaconda3\lib\site-packages\IPython\

In [36]:
hyperparameter_ipynb = dict()

for py_file in tqdm(glob.glob('ipython_files/*.py')):
    
    with open(py_file, 'r',encoding='utf-8') as file:
        file_content = file.read()
                
               
    try:
        for function_name in functionListFromFile:
            
            if str(function_name[0]) not in hyperparameter_ipynb.keys():
                hyperparameter_ipynb[str(function_name[0])]={}
                    
                    
            analyzer= InstantiationAnalyzer(str(function_name[0]))             
            analyzer.parse(file_content)
            
            #print("file content parsed for",py_file, "and " ,str(function_name[0]) )
            
            if(len(analyzer.d.keys())!=0):
                
               # print("key not zero")
                
                for key in analyzer.d.keys():
                    
                    if key not in hyperparameter_ipynb[str(function_name[0])].keys():
                        hyperparameter_ipynb[str(function_name[0])][key]={}
                    
                    for subkey, subval in analyzer.d[key].items():
                        
                                             
                        if subkey in hyperparameter_ipynb[str(function_name[0])][key].keys():
                            hyperparameter_ipynb[str(function_name[0])][key][subkey] += subval
                        else:
                            hyperparameter_ipynb[str(function_name[0])][key][subkey] = subval
                        
                        
    except AttributeError:
        continue    
    

100%|██████████| 9246/9246 [23:33:16<00:00,  9.17s/it]   


In [ ]:
## Saving in pickle file

with open('hyperparameter_ipynb.pickle', 'wb') as handle:
    pickle.dump(hyperparameter_ipynb, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [6]:
## Reading the pickle file
with open('hyperparameter_ipynb.pickle', 'rb') as handle:
    hyperparameter_ipynb = pickle.load(handle)

In [42]:
hyperparameter_ipynb['RandomForestClassifier']['max_depth']

{' int(depth)': 1,
 '10': 18,
 '100': 6,
 '12': 4,
 '14': 1,
 '15': 33,
 '150': 1,
 '16': 3,
 '2': 2,
 '22': 2,
 '25': 3,
 '3': 6,
 '30': 2,
 '4': 4,
 '5': 55,
 '50': 1,
 '6': 10,
 '7': 16,
 '8': 14,
 '9': 1,
 'None': 36,
 'best_m': 1,
 'best_max_depth': 3,
 'depth': 1,
 'g': 2,
 'i': 2,
 'm': 1,
 'max_depth': 3}

## pipeline usage

In [5]:


source = '''from sklearn import svm
    from sklearn.datasets import samples_generator
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import f_regression
    from sklearn.pipeline import Pipeline
    X, y = samples_generator.make_classification(n_informative=5, n_redundant=0, random_state=42)
    anova_filter = SelectKBest(f_regression, k=5)
    clf = svm.SVC(kernel='linear')
    anova_svm.score(X, y)
    anova_svm = Pipeline([('anova', anova_filter), ('svc', clf)])
    anova_svm.set_params(anova__k=10, svc__C=.1).fit(X, y)
    prediction = anova_svm.predict(X)
   
    '''


#The big difference between goto_assignments() and goto_definitions() is that goto_assignments() doesn’t follow imports and statements. 


script = jedi.Script(source,10)
d = script.goto_definitions()[0]

print(d.line)
print(d.description)

print(d.full_name)


29
instance Pipeline
sklearn.pipeline.Pipeline


In [11]:
source = AllContent_py_unique_df.content[0]
pos = len(source.splitlines())
try:
    script = jedi.Script(source,5)
    d = script.goto_definitions()[0]
    print(d.line)
    print(d.description)
    print(d.full_name)

except:
    print("hi")


1
module pandas
pandas


In [85]:
len(AllContent_py_unique_df)

9246

In [4]:
# # Module & Usage analysis 

module_analysis_py_unique = pd.DataFrame(columns=['Filename_index','Description','Package/Module', 'Usage'])
for i in tqdm(range(0, len(AllContent_py_unique_df))):   

    source = AllContent_py_unique_df.content[i]

   
    for pos in range(len(source.splitlines())):
        
        try:
                
            script = jedi.Script(source,pos)
            d = script.goto_definitions()[0]
            if "sklearn" in str(d.full_name):
                
                temp_df = pd.DataFrame([[i,d.description,d.full_name, source.splitlines()[pos-1]]], columns=['Filename_index','Description','Package/Module', 'Usage'])
                module_analysis_py_unique = module_analysis_py_unique.append(temp_df, ignore_index=True)
            
        except:
            continue

            
cols = list(module_analysis_py_unique.columns)
module_analysis_py_unique['count'] = module_analysis_py_unique.groupby(cols)['Filename_index'].transform('size')
module_analysis_py_unique.drop_duplicates(inplace=True)
module_analysis_py_unique.reset_index(drop=True, inplace=True)

100%|███████████████████████████████████████████████████████████████████████████| 2954/2954 [12:24:17<00:00, 15.12s/it]


In [62]:
print(AllContent_py_unique_df.content[5852])


# coding: utf-8

# # Implementing dask-searchCV to speed-up gridsearchCV and including PCA in the pipeline

# This is just a quick notebook to see:
# * How much does dask-searchCV speed things up? (A: More than 3X faster for the 2.TCGA-MLexample)
# * Can you include PCA (and a search for n_components) in the pipeline? (A: Yes!)
# * Are there any limitations to including PCA in the pipeline? (A: It takes a while & my computer couldn't handle n_jobs > 1)
# 

# ## Outline
# 1. Imports, load data, etc.
# 2. Evaluate dask-searchCV on original notebook 2 pipeline (mad feature selection)
#     * SciKit-Learn gridsearchCV (~ 7 minutes)
#     * Dask-searchCV (~ 2 minutes)
#     * Dask-searchCV with cv=10 (
# 3. Evaluate including PCA in the pipeline (note: These instances set n_jobs to 1 because, except for the trivial case, my computer froze when trying to implement with n_jobs set to -1)
#     * Trivial/Benchmark Case [2, 4]
#     * Trivial/Benchmark Case [2, 4] with cv=10
#     * Long list 

In [68]:
import jedi
source = AllContent_py_unique_df.content[5852]    # 231
 

for pos in range(len(source.splitlines())):


    try:
        
        
        
        script = jedi.Script(source,pos)
        d = script.goto_definitions()[0]
        


        #if "sklearn" in str(d.full_name):
        print(source.splitlines()[pos-1], " : ", d.full_name)

#             temp_df = pd.DataFrame([[i,d.description,d.full_name, source.splitlines()[pos-1]]], columns=['Filename_index','Description','Package/Module', 'Usage'])
    
    except:
        continue



    """      :  str
    Get the median absolute deviation (MAD) for each column of x  :  str
    """  :  str
param_grid_original = {  :  dict
}  :  dict
pipeline_original = Pipeline(steps=[  :  list
    ('classify', SGDClassifier(random_state=0, class_weight='balanced'))  :  tuple
param_grid = {  :  dict
}  :  dict
pipeline = Pipeline(steps=[  :  list
    ('classify', SGDClassifier(random_state=0, class_weight='balanced'))  :  tuple
param_grid = {  :  dict
}  :  dict
pipeline = Pipeline(steps=[  :  list
    ('classify', SGDClassifier(random_state=0, class_weight='balanced'))  :  tuple
param_grid = {  :  dict
}  :  dict
pipeline = Pipeline(steps=[  :  list
    ('classify', SGDClassifier(random_state=0, class_weight='balanced'))  :  tuple
param_grid = {  :  dict
}  :  dict
pipeline = Pipeline(steps=[  :  list
    ('classify', SGDClassifier(random_state=0, class_weight='balanced'))  :  tuple
listOfComponents = []  :  list
component = 20  :  int
    component = component + (0.5 * component

In [48]:
pos

252

In [79]:
with open('module_analysis_py_unique.pickle', 'wb') as handle:
    pickle.dump(module_analysis_py_unique, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [6]:
with open('module_analysis_py_unique.pickle', 'rb') as handle:
    module_analysis_py_unique = pickle.load(handle)
   

In [82]:
module_analysis_py_unique[module_analysis_py_unique['Description']=="instance Pipeline"].head(5)

,Filename_index,Description,Package/Module,Usage,count
24,5,instance Pipeline,sklearn.pipeline.Pipeline,"('lr', LinearRegress...",1
25,5,instance Pipeline,sklearn.pipeline.Pipeline,"elmr.fit(xtoy_train, ytoy_train)",1
173,32,instance Pipeline,sklearn.pipeline.Pipeline,pipe = pipeline.Pipeline(steps),1
297,61,instance Pipeline,sklearn.pipeline.Pipeline,('classification'...,1
300,61,instance Pipeline,sklearn.pipeline.Pipeline,('classif...,1


In [8]:
MA_file_py_unique = module_analysis_py_unique['Filename_index'].unique()
                                           

In [29]:
MA_file_py_unique[1098]
                 

1157

## Instantiation analyser on selected files from jedi

In [30]:
hyperparameter_py_unique_after_jedi_Pipeline = dict()
after_jedi_pipeline_files=[]

for i in tqdm(MA_file_py_unique):
    
    
            
    try:
        
            
        analyzer= InstantiationAnalyzer("Pipeline")
        analyzer.parse(AllContent_py_unique_df.content[i])

        if(len(analyzer.d.keys())!=0):
            
            after_jedi_pipeline_files.append(i)

            for key in analyzer.d.keys():
                
                
                

                if key not in hyperparameter_py_unique_after_jedi_Pipeline.keys():
                    hyperparameter_py_unique_after_jedi_Pipeline[key]={}

                for subkey, subval in analyzer.d[key].items():


                    if subkey in hyperparameter_py_unique_after_jedi_Pipeline[key].keys():
                        hyperparameter_py_unique_after_jedi_Pipeline[key][subkey] += subval
                    else:
                        hyperparameter_py_unique_after_jedi_Pipeline[key][subkey] = subval
                        
                        
    except AttributeError:
        continue    
    
    
            

        
    
            

    

100%|██████████████████████████████████████████████████████████████████████████████| 8860/8860 [10:20<00:00, 14.27it/s]


In [31]:
## saving to pickle file
with open('hyperparameter_py_unique_after_jedi_Pipeline.pickle', 'wb') as handle:
    pickle.dump(hyperparameter_py_unique_after_jedi_Pipeline, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
with open('after_jedi_pipeline_files.pickle', 'wb') as handle:
    pickle.dump(after_jedi_pipeline_files, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [36]:
len(after_jedi_pipeline_files)

42

In [39]:
print(AllContent_py_unique_df.content[1])

import numpy as np
import sklearn as sk

from sklearn.neighbors.classification import KNeighborsClassifier


def evaluate(Xtra, ytra, Xtst, ytst, k=1, positive_label=1):
    knn = KNeighborsClassifier(n_neighbors=k, algorithm='brute')
    knn.fit(Xtra, ytra)

    y_true = ytst
    y_pred = knn.predict(Xtst)

    return evaluate_results(y_true, y_pred, positive_label=positive_label)


def evaluate_results(y_true, y_pred, positive_label=1):
    res = y_true == y_pred
    t_gn = float(res.sum()) / len(res)
    mask = y_true != positive_label
    t_mj = float(res[mask].sum()) / len(res[mask])
    mask = y_true == positive_label
    t_mn = float(res[mask].sum()) / len(res[mask])
    # approximation used for instance reduction algorithms
    auc = (1.0 + t_mn - (1.0 - t_mj)) / 2
    return t_gn, t_mj, t_mn, auc


def auc_score(y_true, y_pred, positive_label=1):
    fp_rate, tp_rate, thresholds = sk.metrics.roc_curve(
        y_true, y_pred, pos_label=positive_label)
    return sk.metrics.auc

In [ ]:
make_pipelineQuery = """\
    SELECT repo_name, line
    FROM (
          SELECT DISTINCT(line), repo_name
          FROM(
          SELECT (SPLIT(content, '\\n')) as lines, repo_name, content
          FROM
          `Odyssey_github_sklearn.content_py_unique`
          ), UNNEST(lines) line 

          WHERE (REGEXP_CONTAINS(line, 'make_pipeline')) AND NOT (REGEXP_CONTAINS(line, 'print')) AND NOT (REGEXP_CONTAINS(line, 'import')) AND (REGEXP_CONTAINS(content, 'from sklearn+  import') OR REGEXP_CONTAINS(content, 'import sklearn+ ')) AND %s
         )
     
        GROUP BY 1,2
    
        """ %( excludeByRepoFull(excludedRepos))

    
    
make_pipeline_result = gp.run(make_pipelineQuery)


In [ ]:
np.array(make_pipeline_result)

In [ ]:
sampleQuery = """\
    SELECT sample_repo_name, line,  COUNT(*) count
    FROM (
          SELECT DISTINCT(line), sample_repo_name
          FROM(
          SELECT (SPLIT(content, '\\n')) as lines, sample_repo_name, sample_path
          FROM
          `Odyssey_github_sklearn.content_ipynb`
          ), UNNEST(lines) line 

          WHERE
        (NOT REGEXP_CONTAINS(sample_repo_name, "scikit-learn")) AND
      (NOT REGEXP_CONTAINS(sample_repo_name, "sklearn")) AND
      (NOT REGEXP_CONTAINS(sample_path,"scikit-learn")) AND
      (NOT REGEXP_CONTAINS(sample_path, "sklearn"))
      
         )


          
         GROUP BY 1,2
    

    """ 

sample =gp.run(sampleQuery)

In [ ]:
len(sample)

In [ ]:
sample_df=pd.DataFrame(sample)
sample_df.to_csv("sampleData.csv")